# 0 导入必要的包

In [1]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import ucTools
import funclib
sys.path.append("../../")
import train
sys.path.append("../../")
import test
import time
import joblib
%load_ext autoreload
%autoreload 2

# 1. 读取数据

In [71]:
# 1515 数据
sprot1515 = pd.read_csv('./data/ecoli1515/1515testset_with_unirep.tsv',  sep='\t', index_col=0)

In [91]:
colname =  [['id', 'ec_number', 'seq', 'isemzyme', 'isMultiFunctional', 'blattner_id', 'blattner_ec_number' ] +['f'+str(i) for i in range(1,1901)]]
sprot1515.columns =colname

In [92]:
sprot1515

,id,ec_number,seq,isemzyme,isMultiFunctional,blattner_id,blattner_ec_number,f1,f2,f3,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,P42641,3.6.5.-,MKFVDEASILVVAGDGGNGCVSFRREKYIPKGGPDGGDGGDGGDVW...,True,False,b3183,NaN,0.003056,-0.087184,0.043387,...,0.000332,-0.065542,0.117107,-0.236777,-0.622398,0.012113,0.010506,0.058984,0.061533,0.968014
1,P0AE22,3.1.3.2,MRKITQAISAVCLLFALNSSAVALASSPSPLNPGTNVARLAEQAPI...,True,False,b4055,3.1.3.2,0.001026,0.618393,0.409527,...,0.030307,-0.244636,-0.038585,0.151263,-0.020389,0.095321,-0.031043,-0.881730,-0.196211,0.054455
2,P21499,3.1.13.1,MSQDPFQEREAEKYANPIPSREFILEHLTKREKPASRDELAVELHI...,True,False,b4179,3.1.-.-,0.073632,-0.003258,0.008865,...,0.002123,0.060351,-0.008826,0.006141,-0.129352,-0.075010,0.023712,-0.002246,0.893310,0.140025
3,P39286,3.6.1.-,MSKNKLSKGQQRRVNANHQRRLKTSKEKPDYDDNLFGEPDEGIVIS...,True,False,b4161,3.6.1.-,0.000775,-0.008305,0.041130,...,0.066885,0.020991,0.073577,-0.034962,-0.036755,0.053803,-0.017355,0.100456,-0.202344,0.351100
4,P56100,7.1.1.7,MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI,True,False,b4515,1.10.3.10,0.001811,-0.050181,0.037575,...,0.042016,-0.028950,0.305668,-0.105587,-0.113769,0.680413,0.138797,-0.268528,-0.076093,-0.038212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,P36837,NaN,MNTTTPMGMLQQPRPFFMIFFVELWERFGYYGVQGVLAVFFVKQLG...,False,False,b3496,NaN,0.000070,-0.021424,0.203301,...,0.005911,-0.016855,0.377301,-0.500727,-0.247861,0.752678,-0.005501,0.396191,-0.215832,-0.062812
4746,P77304,NaN,MSTANQKPTESVSLNAFKQPKAFYLIFSIELWERFGYYGLQGIMAV...,False,False,b1634,NaN,0.008628,-0.065382,0.048911,...,-0.001197,-0.101824,0.013007,-0.304450,-0.295525,-0.124492,0.010846,0.032853,-0.155702,-0.183535
4747,P27248,2.1.2.10,MAQQTPLYEQHTLCGARMVDFHGWMMPLHYGSQIDEHHAVRTDAGM...,True,False,b2905,2.1.2.10,0.006723,0.227349,0.029725,...,-0.124573,-0.032099,-0.170996,0.104425,-0.163041,0.183720,-0.038814,0.036993,-0.044558,0.010447
4748,P0AC88,4.2.1.47,MSKVALITGVTGQDGSYLAEFLLEKGYEVHGIKRRASSFNTERVDH...,True,False,b2053,4.2.1.47,0.000665,-0.029179,0.024183,...,0.145032,0.867074,-0.263008,0.009123,0.036783,-0.073805,0.439632,0.014501,-0.205913,-0.005663


In [66]:
sprot1515.iloc[:,8:1909]

,0,1,2,3,4,5,6,7,8,9,...,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899
0,0.003056,-0.087184,0.043387,-0.018850,-0.623890,0.034710,0.133006,-0.056650,-0.004179,-0.002791,...,0.000332,-0.065542,0.117107,-0.236777,-0.622398,0.012113,0.010506,0.058984,0.061533,0.968014
1,0.001026,0.618393,0.409527,-0.002331,-0.670928,0.036821,-0.148798,-0.038413,-0.002616,0.026154,...,0.030307,-0.244636,-0.038585,0.151263,-0.020389,0.095321,-0.031043,-0.881730,-0.196211,0.054455
2,0.073632,-0.003258,0.008865,-0.012720,-0.072697,-0.026430,-0.033917,-0.000300,-0.001022,0.041117,...,0.002123,0.060351,-0.008826,0.006141,-0.129352,-0.075010,0.023712,-0.002246,0.893310,0.140025
3,0.000775,-0.008305,0.041130,-0.001737,-0.156966,0.009515,0.400612,-0.021405,-0.002408,0.028499,...,0.066885,0.020991,0.073577,-0.034962,-0.036755,0.053803,-0.017355,0.100456,-0.202344,0.351100
4,0.001811,-0.050181,0.037575,-0.005655,0.025414,0.003502,-0.096007,-0.006303,-0.000473,0.012226,...,0.042016,-0.028950,0.305668,-0.105587,-0.113769,0.680413,0.138797,-0.268528,-0.076093,-0.038212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,0.000070,-0.021424,0.203301,0.025054,0.425925,-0.013487,-0.131236,-0.000778,-0.000883,-0.099993,...,0.005911,-0.016855,0.377301,-0.500727,-0.247861,0.752678,-0.005501,0.396191,-0.215832,-0.062812
4746,0.008628,-0.065382,0.048911,-0.215166,-0.197027,0.001949,0.045984,-0.001380,-0.004231,0.022663,...,-0.001197,-0.101824,0.013007,-0.304450,-0.295525,-0.124492,0.010846,0.032853,-0.155702,-0.183535
4747,0.006723,0.227349,0.029725,0.032317,0.064065,0.092823,-0.044260,-0.024524,-0.001700,0.055556,...,-0.124573,-0.032099,-0.170996,0.104425,-0.163041,0.183720,-0.038814,0.036993,-0.044558,0.010447
4748,0.000665,-0.029179,0.024183,-0.005963,-0.175254,0.042627,0.554610,-0.020871,-0.008380,0.105102,...,0.145032,0.867074,-0.263008,0.009123,0.036783,-0.073805,0.439632,0.014501,-0.205913,-0.005663


In [4]:
funclib.table2fasta(sprot1515, './data/sprot1515.fasta')

Write finished


## 2. 序列比对

##   -------------

In [654]:
file_1515 = './data/ecoli1515/1515testset_with_unirep.tsv'
data_1515 = pd.read_csv(file_1515, sep='\t', index_col=0)

In [298]:
file_1515 = './data/ecoli1515/1515testset_with_unirep.tsv'
data_1515 = pd.read_csv(file_1515, sep='\t', index_col=0)
colname =  [['id', 'ec_number', 'seq', 'isemzyme', 'isMultiFunctional', 'blattner_id', 'blattner_ec_number' ] +['f'+str(i) for i in range(1,1901)]]
data_1515.columns =colname
test_1515 = data_1515[['id', 'ec_number', 'blattner_ec_number']]

In [645]:
data_1515

id ec_number                                                seq  \
0     P42641   3.6.5.-  MKFVDEASILVVAGDGGNGCVSFRREKYIPKGGPDGGDGGDGGDVW...   
1     P0AE22   3.1.3.2  MRKITQAISAVCLLFALNSSAVALASSPSPLNPGTNVARLAEQAPI...   
2     P21499  3.1.13.1  MSQDPFQEREAEKYANPIPSREFILEHLTKREKPASRDELAVELHI...   
3     P39286   3.6.1.-  MSKNKLSKGQQRRVNANHQRRLKTSKEKPDYDDNLFGEPDEGIVIS...   
4     P56100   7.1.1.7              MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI   
...      ...       ...                                                ...   
4745  P36837       NaN  MNTTTPMGMLQQPRPFFMIFFVELWERFGYYGVQGVLAVFFVKQLG...   
4746  P77304       NaN  MSTANQKPTESVSLNAFKQPKAFYLIFSIELWERFGYYGLQGIMAV...   
4747  P27248  2.1.2.10  MAQQTPLYEQHTLCGARMVDFHGWMMPLHYGSQIDEHHAVRTDAGM...   
4748  P0AC88  4.2.1.47  MSKVALITGVTGQDGSYLAEFLLEKGYEVHGIKRRASSFNTERVDH...   
4749  P0A6N8       NaN  MPRANEIKKGMVLNYNGKLLLVKDIDIQSPTARGAATLYKMRFSDV...   

     isemzyme isMultiFunctional blattner_id blattner_ec_number        f1  \
0        True             False       b3183                NaN  0.003056   
1        True             False       b4055            3.1.3.2  0.001026   
2        True             False       b4179            3.1.-.-  0.073632   
3        True             False       b4161            3.6.1.-  0.000775   
4        True             False       b4515          1.10.3.10  0.001811   
...       ...               ...         ...                ...       ...   
4745    False             False       b3496                NaN  0.000070   
4746    False             False       b1634                NaN  0.008628   
4747     True             False       b2905           2.1.2.10  0.006723   
4748     True             False       b2053           4.2.1.47  0.000665   
4749    False             False       b2171                NaN  0.000589   

            f2        f3        f4        f5        f6        f7        f8  \
0    -0.087184  0.043387 -0.018850 -0.623890  0.034710  0.133006 -0.056650   
1     0.618393  0.409527 -0.002331 -0.670928  0.036821 -0.148798 -0.038413   
2    -0.003258  0.008865 -0.012720 -0.072697 -0.026430 -0.033917 -0.000300   
3    -0.008305  0.041130 -0.001737 -0.156966  0.009515  0.400612 -0.021405   
4    -0.050181  0.037575 -0.005655  0.025414  0.003502 -0.096007 -0.006303   
...        ...       ...       ...       ...       ...       ...       ...   
4745 -0.021424  0.203301  0.025054  0.425925 -0.013487 -0.131236 -0.000778   
4746 -0.065382  0.048911 -0.215166 -0.197027  0.001949  0.045984 -0.001380   
4747  0.227349  0.029725  0.032317  0.064065  0.092823 -0.044260 -0.024524   
4748 -0.029179  0.024183 -0.005963 -0.175254  0.042627  0.554610 -0.020871   
4749  0.032014  0.013608 -0.008443 -0.115612  0.028364 -0.153695 -0.032118   

            f9       f10       f11       f12       f13       f14       f15  \
0    -0.004179 -0.002791 -0.002462  0.002599  0.048714  0.542259  0.009463   
1    -0.002616  0.026154  0.002405  0.001509  0.159823 -0.003070  0.004941   
2    -0.001022  0.041117 -0.046263  0.002500  0.003450 -0.647001  0.000169   
3    -0.002408  0.028499  0.002506  0.000343  0.094061  0.207324  0.066717   
4    -0.000473  0.012226 -0.007325  0.000229  0.079678  0.055803  0.013916   
...        ...       ...       ...       ...       ...       ...       ...   
4745 -0.000883 -0.099993  0.836958  0.000932  0.031655 -0.030638  0.086026   
4746 -0.004231  0.022663  0.821562 -0.000184  0.113237 -0.018413  0.045726   
4747 -0.001700  0.055556 -0.014932 -0.004102  0.006636 -0.052259  0.001718   
4748 -0.008380  0.105102  0.008972 -0.000733  0.003622  0.059358 -0.007454   
4749 -0.000680  0.051763 -0.121068 -0.000767  0.050263  0.110978  0.005803   

           f16       f17       f18       f19       f20       f21       f22  \
0    -0.000082 -0.009477 -0.090929  0.357896 -0.001747 -0.006085 -0.301930   
1     0.001484  0.026647 -0.172789  0.213775 -0.003496 -0.020142 -0.319490   
2     0.000002  0.045165  0.002784 -0.964551 -0.014308 -0.000986

In [656]:
len(set(data_1515.id))

4427

In [300]:
set(test_1515.id)

{('id',)}

In [97]:
res_table_head = ['id', 'sseqid', 'pident', 'length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore']
blast_res = pd.read_csv('../../results/1515/balastres.tsv', sep='\t', names=res_table_head)

In [208]:
uctools =  ucTools.ucTools('172.16.25.20')
cnx_mimic = uctools.db_conn()
sql ='''SELECT id, isemzyme, "isMultiFunctional","functionCounts", ec_number, ec_specific_level from tb_sprot where id in ('{0}') '''.format('\',\''.join(blast_res.sseqid.values))
blast_tags = pd.read_sql_query(sql,cnx_mimic)
blast_final = blast_res.merge(blast_tags, left_on='sseqid', right_on='id', how='left')

blast_final.drop_duplicates(subset=['id_x'], keep='first', inplace =True)
blast_final=blast_final[['id_x', 'pident', 'isemzyme', 'isMultiFunctional', 'functionCounts', 'ec_number']]
blast_final=blast_final.rename(columns={'id_x':'id', 'isemzyme':'isemzyme_blast', 'isMultiFunctional':'isMultiFunctional_blast', 'functionCounts':'functionCounts_blast', 'ec_number':'ec_number_blast'})
blast_final.reset_index(drop=True, inplace=True)

In [644]:
blast_final

,id,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast
0,P42641,100.0,True,False,1,3.6.5.-
1,P0AE22,100.0,True,False,1,3.1.3.2
2,P21499,100.0,True,False,1,3.1.13.1
3,P39286,100.0,True,False,1,3.6.1.-
4,P56100,100.0,True,False,1,7.1.1.7
...,...,...,...,...,...,...
4317,P36837,100.0,False,False,1,-
4318,P77304,100.0,False,False,1,-
4319,P27248,100.0,True,False,1,2.1.2.10
4320,P0AC88,100.0,True,False,1,4.2.1.47


In [227]:
test_1515 = data_1515[['id', 'ec_number', 'blattner_ec_number']]
test_1515.drop_duplicates(subset=None, keep='first', inplace =True)
test_1515.reset_index(drop=True, inplace=True)

<ipython-input-227-838251431452>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_1515.drop_duplicates(subset=None, keep='first', inplace =True)


In [254]:
file_1515 = './data/ecoli1515/1515testset_with_unirep.tsv'
data_1515 = pd.read_csv(file_1515, sep='\t', index_col=0)

In [272]:
test_1515.drop_duplicates(subset=None, keep='first', inplace =True)
data_test_blast_res = test_1515.merge(blast_final, on=['id'], how='left')

<ipython-input-272-e805fd579c7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_1515.drop_duplicates(subset=None, keep='first', inplace =True)


In [279]:
data_test_blast_res

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7
...,...,...,...,...,...,...,...,...
4422,P36837,NaN,NaN,100.0,False,False,1.0,-
4423,P77304,NaN,NaN,100.0,False,False,1.0,-
4424,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10
4425,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47


In [294]:
def get_isEnzymeRes(querydata, model_file):
    """[获取「酶｜非酶」预测结果]
    Args:
        querydata ([DataFrame]): [需要预测的数据]
        model_file ([string]): [模型文件]
    Returns:
        [DataFrame]: [预测结果、预测概率]
    """
    model = joblib.load(model_file)
    predict = model.predict(querydata)
    predictprob = model.predict_proba(querydata)
    return predict, predictprob[:,1]

In [350]:
isEnzyme_model = './model/isenzyme.model'
isEnzyme_pred, isEnzyme_pred_prob = get_isEnzymeRes(querydata=data_1515.iloc[:,8:1909], model_file=isEnzyme_model )

In [351]:
isEnzyme_pred = pd.DataFrame(isEnzyme_pred)
isEnzyme_pred['isenzyme_slice_prob'] = isEnzyme_pred_prob
isEnzyme_pred['id']=data_1515.id
isEnzyme_pred.rename(columns={0:'isenzyme_slice_pred'}, inplace=True)

In [352]:
isEnzyme_pred.drop_duplicates(subset=None, keep='first', inplace =True)

In [353]:
data_test_blast_isemzyme_res = data_test_blast_res.merge(isEnzyme_pred, on='id', how='left')

In [355]:
data_test_blast_isemzyme_res

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1,0.988359
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1,0.586609
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,1,0.659836
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1,0.940199
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,0,0.088895
...,...,...,...,...,...,...,...,...,...,...
4422,P36837,NaN,NaN,100.0,False,False,1.0,-,0,0.002424
4423,P77304,NaN,NaN,100.0,False,False,1.0,-,0,0.001840
4424,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,1,0.983754
4425,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47,1,0.918800


In [358]:
result_slice = pd.read_csv('../../results/1515/slice_results.txt',  header = None, skiprows=1 ,sep=' ')

In [359]:
result_slice

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,978:1.32985,1251:1.1761,1875:1.12359,1981:1.13307,1983:1.13258,2032:1.15958,2820:1.13579,4178:1.40714,4179:1.19324,5134:1.12219,5148:1.12175,5151:1.1245,5152:1.13327,5165:1.14828,5171:1.12431,5188:1.12921,5194:1.13408,5244:1.19891,5277:1.13309,5350:1.15252
1,68:1.22049,384:1.20907,899:1.20413,964:1.20556,2232:1.21264,2445:1.21733,2537:1.21288,2541:1.24548,2595:1.22516,2777:1.23049,3190:1.20577,3297:1.59273,3379:1.2052,3823:1.21487,3940:1.23014,3989:1.20857,3991:1.21159,4492:1.21879,4804:1.21204,4823:1.21933
2,1581:1.14419,1618:1.15876,1619:1.15593,1638:1.13789,1725:1.17049,1771:1.19192,1819:1.19331,2036:1.14335,2164:1.13019,2701:1.13465,2723:1.17565,2920:1.1754,2923:1.18286,2980:1.19516,3172:1.30745,3262:1.20846,3418:1.14923,4180:1.13491,4907:1.31918,5097:1.18461
3,1981:1.16911,1983:1.1685,2163:1.1533,2536:1.16727,2705:1.17472,2729:1.16009,2820:1.1686,2879:1.17464,2926:1.15614,2927:1.16013,2931:1.20865,2942:1.15794,3079:1.15491,3080:1.17788,3083:1.15213,3084:1.16575,3088:1.15254,3127:1.37738,5165:1.15368,5267:1.19377
4,28:1.1214,956:1.14524,991:1.11139,1479:1.11841,1710:1.24343,1820:1.17987,2160:1.11292,2254:1.11223,2531:1.14813,3264:1.11437,3342:1.21598,3576:1.10494,3586:1.10919,4501:1.15548,4763:1.1433,4782:1.14731,5306:1.27891,5309:1.26312,5311:1.18409,5341:1.14645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,364:1.10582,1820:1.09455,2183:1.10275,2802:1.13879,2844:1.10321,2871:1.09348,2966:1.09373,2995:1.09578,3036:1.09462,3642:1.10021,4094:1.10546,4206:1.12461,4229:1.08915,5244:1.08593,5311:1.09403,5321:1.23236,5324:1.11967,5325:1.12331,5333:1.12135,5337:1.10606
4746,1107:1.04959,1124:1.04284,1504:1.05272,1510:1.06673,1511:1.04922,1820:1.04942,2017:1.12438,2227:1.08629,2802:1.11666,2844:1.04493,3313:1.05197,4135:1.08846,4229:1.0422,4346:1.05868,4970:1.06368,5225:1.0561,5321:1.26276,5324:1.04693,5333:1.04279,5337:1.15823
4747,1683:1.19604,1832:1.68723,2019:1.18643,2441:1.18262,2442:1.18003,2597:1.18993,2896:1.22555,2988:1.18051,3117:1.19074,4227:1.22049,4322:1.18885,4468:1.18771,4487:1.18138,4786:1.1824,4832:1.21894,4849:1.19381,4920:1.1937,4934:1.2164,5106:1.19829,5295:1.21945
4748,122:1.16586,130:1.16973,191:1.12869,615:1.13622,628:1.14631,1053:1.16233,1594:1.13349,2268:1.14384,2377:1.21918,2426:1.15232,2750:1.14193,4158:1.13906,4184:1.14266,4222:1.14098,4299:1.15003,4382:1.1392,4479:1.60657,4861:1.13899,4872:1.13504,5207:1.14459


In [360]:
def sort_results(result_slice):
    """
    将slice的实验结果排序，并按照推荐顺序以两个矩阵的形式返回
    @pred_top：预测结果排序
    @pred_pb_top：预测结果评分排序
    """
    pred_top =[]
    pred_pb_top =[]
    aac =[]
    for index, row in result_slice.iterrows():
        row_trans= [*row.apply(lambda x: x.split(':')).values]
        row_trans = pd.DataFrame(row_trans).sort_values(by=[1], ascending=False)
        pred_top += [list(np.array(row_trans[0]).astype('int'))]
        pred_pb_top += [list(np.array(row_trans[1]).astype('float'))]
    pred_top = pd.DataFrame(pred_top)
    pred_pb_top = pd.DataFrame(pred_pb_top)
    return pred_top,  pred_pb_top

In [361]:
slice_pred_top, slice_pred_top_pb = sort_results(result_slice)

In [362]:
slice_pred_top

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,4178,978,5244,4179,1251,2032,5350,5165,2820,5194,5152,5277,1981,1983,5188,5151,5171,1875,5134,5148
1,3297,2541,2777,3940,2595,68,4823,4492,2445,3823,2537,2232,4804,3991,384,3989,3190,964,3379,899
2,4907,3172,3262,2980,1819,1771,5097,2923,2723,2920,1725,1618,1619,3418,1581,2036,1638,4180,2701,2164
3,3127,2931,5267,3080,2705,2879,1981,2820,1983,2536,3084,2927,2729,2942,2926,3079,5165,2163,3088,3083
4,5306,5309,1710,3342,5311,1820,4501,2531,4782,5341,956,4763,28,1479,3264,2160,2254,991,3586,3576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,5321,2802,4206,5325,5333,5324,5337,364,4094,2844,2183,3642,2995,3036,1820,5311,2966,2871,4229,5244
4746,5321,5337,2017,2802,4135,2227,1510,4970,4346,5225,1504,3313,1107,1820,1511,5324,2844,1124,5333,4229
4747,1832,2896,4227,5295,4832,4934,5106,1683,4849,4920,3117,2597,4322,4468,2019,2441,4786,4487,2988,2442
4748,4479,2377,130,122,1053,2426,4299,628,5207,2268,4184,2750,4222,4382,4158,4861,615,4872,1594,191


In [422]:
dict_label_ec= np.load('../../data/dict_label_ec.npy', allow_pickle=True).item()

In [423]:
slice_pred_ec_top=pd.DataFrame()
for i in range(slice_pred_top.shape[1]):
    slice_pred_ec_top[i]= slice_pred_top[i].apply(lambda x: dict_label_ec.get(x))

In [424]:
slice_pred_ec_top['id'] = data_1515.id

In [425]:
slice_pred_ec_top.drop_duplicates(subset=None, keep='first', inplace =True)

In [419]:
colname =  ['stop'+str(i) for i in range(1,21)] +['id']
slice_pred_ec_top.columns =list(colname)

In [426]:
slice_pred_ec_top.id

0       P42641
1       P0AE22
2       P21499
3       P39286
4       P56100
         ...  
4745    P36837
4746    P77304
4747    P27248
4748    P0AC88
4749    P0A6N8
Name: id, Length: 4427, dtype: object

In [427]:
data_test_blast_isemzyme_slice_res = data_test_blast_isemzyme_res.merge(slice_pred_ec_top, on='id', how='left')

In [428]:
data_test_blast_isemzyme_slice_res

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,...,10,11,12,13,14,15,16,17,18,19
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1,0.988359,...,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1,0.586609,...,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,1,0.659836,...,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1,0.940199,...,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,0,0.088895,...,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4422,P36837,NaN,NaN,100.0,False,False,1.0,-,0,0.002424,...,2.4.1.17,3.4.16.5,2.7.7.74,2.7.8.34,2.1.1.86,7.1.1.9,2.7.7.41,2.7.1.66,4.1.1.111,6.3.3.3
4423,P77304,NaN,NaN,100.0,False,False,1.0,-,0,0.001840,...,1.8.7.3,3.1.3.4,1.21.99.4,2.1.1.86,1.8.98.6,7.2.2.13,2.7.13.3,1.2.5.3,7.2.2.4,4.1.1.111
4424,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,1,0.983754,...,2.8.3.9,2.5.1.94,4.1.2.42,4.2.1.33,2.3.1.277,2.4.2.54,4.4.1.24,4.2.1.55,2.7.7.68,2.4.2.57
4425,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47,1,0.918800,...,3.7.1.12,2.7.1.157,4.1.1.105,4.1.99.5,3.6.1.64,5.1.3.15,1.14.13.25,5.1.3.26,2.1.1.179,1.1.1.308


In [444]:
aa['isemzyme_blast'] = aa['isemzyme_blast'].astype('str')
aa['isenzyme_slice_pred'] = aa['isenzyme_slice_pred'].apply(lambda x: 'True' if x==1 else 'False')


In [429]:
aa=data_test_blast_isemzyme_slice_res

In [446]:
pd.set_option('display.max_columns', None)

In [542]:
#完全一致的
aa['emzyme_pred_final']=pd.DataFrame(list_enzyme_pred)[0]
aa['emzyme_pred_final_prob']=pd.DataFrame(list_enzyme_pred)[1]

In [534]:
list_enzyme_pred = []
for index, row in aa.iterrows():
    list_enzyme_pred+=[get_emzyme_rep(row.isemzyme_blast, row.isenzyme_slice_pred, row.isenzyme_slice_prob)]

In [526]:
def get_emzyme_rep(balstpred, slicepred, sliceprob):
    if str(balstpred) == str(slicepred):
        return [balstpred,1.0]
    if (str(balstpred) == 'uncertain') & (str(slicepred)=='True'):
        return [slicepred, 1.0]
    if str(balstpred) == 'True':
        return [balstpred, 0.9]
    if str(balstpred) == 'uncertain':
        return ['True', 0.8 ]
    if (str(balstpred) == 'nan') | (str(balstpred) == 'NaN'):
        return [slicepred, 0.9*sliceprob]
    else: 
        return [slicepred, 0.5*sliceprob]

In [525]:
aa[aa.emzyme_pred_final.isnull()]

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,emzyme_pred_final
14,P37690,NaN,NaN,100.0,False,False,1.0,-,True,0.880858,3.4.24.75,3.4.14.13,1.5.99.13,4.3.1.27,2.3.1.157,2.3.1.203,3.4.21.22,2.7.7.23,3.4.21.7,3.2.1.81,3.4.21.68,2.3.1.210,3.4.21.2,3.4.21.4,3.4.21.109,3.1.1.65,3.4.17.13,1.14.15.15,3.4.22.32,2.7.1.45,None
28,P77737,NaN,NaN,100.0,False,False,1.0,-,True,0.727023,7.4.2.9,7.4.2.10,2.7.1.21,7.2.2.11,7.6.2.14,7.3.2.2,1.2.7.8,1.12.98.4,4.2.99.18,3.6.1.72,3.6.1.71,1.3.1.2,7.2.2.2,3.1.12.1,4.1.99.23,1.8.98.6,1.12.98.1,1.2.7.7,1.3.5.1,1.4.99.5,None
118,P52130,NaN,NaN,100.0,False,False,1.0,-,True,0.635598,3.5.4.16,2.7.7.6,4.1.1.50,2.7.4.6,2.1.1.148,5.5.1.6,3.1.21.4,2.3.1.204,1.17.4.1,2.7.7.7,1.16.3.1,2.3.1.133,1.12.1.2,3.1.4.37,2.3.1.33,5.4.99.43,3.6.1.58,2.7.7.96,5.4.99.29,3.6.1.64,None
124,P76555,2.7.2.1,NaN,100.0,False,False,1.0,-,True,0.896631,4.1.1.65,3.5.3.26,5.3.1.8,4.1.1.11,2.4.2.15,4.2.1.108,2.4.2.4,2.4.2.2,4.1.99.20,2.4.2.3,1.13.11.53,4.4.1.3,2.7.1.23,2.3.1.156,1.13.11.24,3.4.11.19,3.7.1.2,4.2.1.119,2.4.1.230,4.2.1.141,None
129,P10346,NaN,NaN,100.0,False,False,1.0,-,True,0.661911,7.3.2.1,7.6.2.15,7.4.2.1,7.4.2.12,7.4.2.7,7.3.2.2,7.6.2.14,7.6.2.12,7.6.2.3,7.6.2.2,7.2.2.2,7.2.2.11,7.6.2.7,7.2.2.16,7.5.2.8,7.5.2.11,7.5.2.9,7.6.2.8,7.6.2.13,2.7.8.12,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4380,P36928,NaN,NaN,100.0,False,False,1.0,-,True,0.778177,2.7.1.33,2.7.2.7,1.3.7.8,2.3.1.234,2.7.1.30,2.7.1.14,3.1.22.4,2.7.1.58,2.5.1.131,2.7.1.1,2.7.1.2,2.7.1.188,2.7.1.17,2.7.1.63,2.7.1.53,3.2.1.183,2.7.2.1,2.7.1.51,3.5.1.124,2.7.1.215,None
4381,P52123,NaN,NaN,100.0,False,False,1.0,-,True,0.621218,4.3.2.1,2.5.1.1,3.1.26.3,1.3.2.3,2.7.7.19,2.5.1.10,2.1.1.218,3.4.22.45,2.3.2.23,2.4.1.198,3.1.7.10,2.7.7.42,2.7.1.158,4.2.3.54,4.1.99.13,3.4.22.68,4.2.3.75,2.7.7.89,6.4.1.2,4.2.3.76,None
4388,Q2EER5,NaN,NaN,100.0,False,False,1.0,-,True,0.533342,1.1.1.37,2.1.1.113,2.7.1.197,2.7.1.191,1.8.99.2,6.5.1.2,1.16.1.7,2.7.7.2,2.1.1.67,4.3.1.17,2.4.2.9,5.4.99.18,1.18.1.2,2.7.4.27,2.7.11.32,1.14.19.9,2.7.1.205,2.1.1.211,5.3.1.34,1.20.4.1,None
4397,P64612,NaN,NaN,100.0,False,False,1.0,-,True,0.500412,7.1.2.2,3.6.4.12,3.6.4.6,2.7.1.21,6.6.1.1,7.2.2.1,2.3.1.193,3.1.3.32,3.1.21.5,2.4.1.141,6.6.1.2,7.4.2.4,2.7.4.13,2.7.1.163,2.1.1.160,3.1.21.3,1.4.1.17,1.14.14.8,3.4.22.44,2.3.1.38,None


In [460]:
aa[(~aa.emzyme_pred_final.isnull()) & (aa.emzyme_pred_final=='True')]

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,emzyme_pred_final
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,True,0.988359,3.6.5.2,1.18.6.1,6.3.3.3,3.6.5.3,1.3.7.7,2.3.1.292,7.3.2.7,6.2.1.46,2.7.1.22,6.3.1.5,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27,True
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,True,0.586609,3.1.3.2,2.5.1.31,2.7.1.180,3.5.1.1,2.5.1.92,1.11.1.5,4.99.1.3,4.2.1.66,2.4.2.6,3.4.22.8,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,True
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,True,0.659836,5.2.1.8,3.1.13.1,3.1.26.12,2.7.7.6,2.1.1.85,2.1.1.359,5.6.2.1,2.7.4.24,2.7.11.26,2.7.4.21,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136,True
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,True,0.940199,2.9.1.3,2.7.4.33,6.3.4.5,2.8.2.20,2.7.1.113,2.7.1.76,2.3.1.239,2.7.1.22,2.3.1.240,2.5.1.27,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23,True
5,P23874,2.7.11.1,2.7.11.1,100.0,True,False,1.0,2.7.11.1,True,0.980370,6.1.1.14,2.5.1.18,2.3.1.181,2.7.7.12,2.3.1.204,2.7.7.47,6.1.1.2,1.13.11.80,1.14.11.45,6.3.1.12,5.4.99.30,2.5.1.10,2.5.1.1,5.3.1.9,5.4.99.43,3.8.1.10,2.1.1.200,1.14.13.69,3.5.99.2,1.14.13.172,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4417,P16676,7.3.2.3,3.6.3.25,100.0,True,False,1.0,7.3.2.3,True,0.968628,7.3.2.3,7.6.2.11,7.6.2.10,7.2.2.7,7.5.2.1,7.6.2.16,7.3.2.5,7.3.2.6,1.7.1.16,2.7.8.11,3.1.4.56,7.4.2.10,4.6.1.1,7.6.2.14,4.2.1.69,3.1.3.43,1.4.99.5,1.13.12.21,2.7.1.221,3.4.24.15,True
4420,P76330,2.7.7.65,2.7.7.65,100.0,True,False,1.0,2.7.7.65,True,0.881468,2.7.7.65,4.1.1.81,4.1.1.111,2.1.1.1,2.7.7.72,7.2.2.7,2.3.1.60,7.3.2.4,2.3.1.36,2.1.1.315,2.7.13.3,1.14.15.13,2.3.1.267,2.7.8.44,7.6.2.8,3.1.4.59,2.1.1.99,2.3.1.178,3.4.23.2,2.1.1.156,True
4421,P36655,1.8.1.8,1.8.1.8,100.0,True,False,1.0,1.8.1.8,True,0.788530,1.8.1.8,1.11.1.22,1.11.1.24,1.11.1.29,1.11.1.9,1.8.4.9,1.11.1.12,1.11.1.25,1.11.1.17,1.20.4.1,1.20.4.3,1.11.1.27,1.8.4.2,3.1.1.76,3.4.11.5,3.5.4.38,3.1.1.45,3.4.21.103,3.8.1.3,5.1.3.2,True
4424,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,True,0.983754,2.1.2.10,2.7.2.11,4.1.1.11,6.5.1.4,5.1.1.1,5.3.1.6,6.1.1.10,2.1.1.269,5.1.1.7,5.3.1.26,2.8.3.9,2.5.1.94,4.1.2.42,4.2.1.33,2.3.1.277,2.4.2.54,4.4.1.24,4.2.1.55,2.7.7.68,2.4.2.57,True


In [467]:
aa[(aa.emzyme_pred_final.isnull()) ]

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,emzyme_pred_final
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,False,0.088895,7.1.1.2,7.1.1.7,2.1.1.295,3.1.3.78,7.1.1.9,2.1.1.86,4.2.1.78,2.5.1.21,4.4.1.20,7.2.4.2,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5,NaN
9,Q46865,3.1.-.-,3.1.-.-,100.0,uncertain,False,1.0,3.1.-.-,False,0.060774,2.7.8.7,3.5.1.135,4.4.1.26,3.1.13.5,1.14.11.45,4.2.3.12,3.1.21.4,4.2.1.154,5.1.99.8,2.5.1.109,3.6.1.56,5.3.3.4,3.5.1.5,4.2.1.104,3.5.4.35,2.5.1.107,4.1.3.40,1.14.11.33,1.14.11.64,4.2.1.94,NaN
10,P0AE70,3.1.27.-,3.1.-.-,100.0,True,False,1.0,3.1.27.-,False,0.120603,2.6.1.42,3.6.1.29,1.14.11.40,3.1.1.29,4.2.1.96,2.3.2.34,3.1.2.28,2.7.7.56,6.3.1.17,2.5.1.18,3.2.2.17,6.3.2.41,3.4.19.15,5.4.99.43,2.4.1.16,2.4.2.27,5.1.3.15,4.3.2.9,2.7.1.26,3.6.1.67,NaN
14,P37690,NaN,NaN,100.0,False,False,1.0,-,True,0.880858,3.4.24.75,3.4.14.13,1.5.99.13,4.3.1.27,2.3.1.157,2.3.1.203,3.4.21.22,2.7.7.23,3.4.21.7,3.2.1.81,3.4.21.68,2.3.1.210,3.4.21.2,3.4.21.4,3.4.21.109,3.1.1.65,3.4.17.13,1.14.15.15,3.4.22.32,2.7.1.45,NaN
17,Q46896,3.1.-.-,3.1.-.-,100.0,uncertain,False,1.0,3.1.-.-,True,0.704433,4.1.1.65,5.4.99.30,2.7.11.5,4.3.2.9,1.8.5.7,4.2.1.9,2.3.1.228,3.4.13.19,3.5.2.2,1.13.12.19,1.14.20.7,5.3.99.6,4.6.1.22,1.4.7.1,4.2.99.23,2.4.2.30,1.14.11.39,1.14.11.17,3.6.1.25,3.4.21.113,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4399,P33368,1.-.-.-,1.-.-.-,100.0,uncertain,False,1.0,1.-.-.-,True,0.988147,1.3.1.9,1.1.1.119,1.1.1.295,1.1.1.206,1.1.1.173,1.5.1.33,1.1.1.236,1.1.1.47,1.1.1.159,1.1.1.53,1.3.1.29,1.3.1.56,1.3.1.60,1.2.1.8,1.1.1.377,3.4.17.13,2.3.1.46,1.1.1.233,1.1.1.331,1.1.1.62,NaN
4402,P0DSG3,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.015993,7.1.1.2,4.1.3.40,1.14.13.105,2.5.1.103,3.6.1.67,5.3.1.32,1.14.99.48,3.4.22.39,2.3.2.6,2.3.1.135,1.14.11.33,5.4.99.20,5.1.99.7,1.3.7.6,4.4.1.17,3.1.12.1,3.1.21.4,3.6.1.55,3.5.1.135,3.3.2.11,NaN
4404,P0DSG9,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.062944,1.14.13.81,3.4.17.6,7.1.1.2,1.3.7.6,3.1.21.4,3.6.1.55,4.1.3.40,3.3.2.11,2.4.1.132,2.4.1.257,2.3.2.34,2.5.1.103,6.3.1.20,3.2.2.21,3.6.1.67,4.4.1.20,7.2.4.2,6.1.1.20,1.3.7.3,4.3.2.9,NaN
4405,P76561,3.1.-.-,3.1.-.-,100.0,uncertain,False,1.0,3.1.-.-,True,0.968870,6.1.1.19,6.3.1.18,3.4.14.5,1.14.11.48,2.1.1.260,3.5.1.26,1.3.5.6,6.3.1.19,1.13.11.38,2.3.1.93,3.1.1.6,2.4.1.376,6.3.1.11,2.4.2.63,2.7.4.1,3.1.3.62,3.1.3.80,1.14.12.25,3.1.1.112,3.6.1.45,NaN


In [630]:
aa.to_excel('../../results/1515/1515_final_pred.xlsx', index=None)

In [463]:
aa['emzyme_pred_final'] = aa[(aa.isemzyme_blast=='uncertain') & (aa.isenzyme_slice_pred=='True')].isenzyme_slice_pred

In [545]:
aa

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,emzyme_pred_final,emzyme_pred_final_prob
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,True,0.988359,3.6.5.2,1.18.6.1,6.3.3.3,3.6.5.3,1.3.7.7,2.3.1.292,7.3.2.7,6.2.1.46,2.7.1.22,6.3.1.5,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27,True,1.0
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,True,0.586609,3.1.3.2,2.5.1.31,2.7.1.180,3.5.1.1,2.5.1.92,1.11.1.5,4.99.1.3,4.2.1.66,2.4.2.6,3.4.22.8,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,True,1.0
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,True,0.659836,5.2.1.8,3.1.13.1,3.1.26.12,2.7.7.6,2.1.1.85,2.1.1.359,5.6.2.1,2.7.4.24,2.7.11.26,2.7.4.21,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136,True,1.0
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,True,0.940199,2.9.1.3,2.7.4.33,6.3.4.5,2.8.2.20,2.7.1.113,2.7.1.76,2.3.1.239,2.7.1.22,2.3.1.240,2.5.1.27,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23,True,1.0
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,False,0.088895,7.1.1.2,7.1.1.7,2.1.1.295,3.1.3.78,7.1.1.9,2.1.1.86,4.2.1.78,2.5.1.21,4.4.1.20,7.2.4.2,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5,True,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4422,P36837,NaN,NaN,100.0,False,False,1.0,-,False,0.002424,7.2.2.10,2.7.1.202,3.8.1.3,7.2.2.14,7.2.2.4,7.2.2.13,7.2.2.8,1.12.98.3,3.5.4.4,2.7.13.3,2.4.1.17,3.4.16.5,2.7.7.74,2.7.8.34,2.1.1.86,7.1.1.9,2.7.7.41,2.7.1.66,4.1.1.111,6.3.3.3,False,1.0
4423,P77304,NaN,NaN,100.0,False,False,1.0,-,False,0.001840,7.2.2.10,7.2.2.8,2.3.1.275,2.7.1.202,3.6.1.27,2.4.1.227,1.8.98.5,5.3.99.7,4.1.3.16,6.3.2.45,1.8.7.3,3.1.3.4,1.21.99.4,2.1.1.86,1.8.98.6,7.2.2.13,2.7.13.3,1.2.5.3,7.2.2.4,4.1.1.111,False,1.0
4424,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,True,0.983754,2.1.2.10,2.7.2.11,4.1.1.11,6.5.1.4,5.1.1.1,5.3.1.6,6.1.1.10,2.1.1.269,5.1.1.7,5.3.1.26,2.8.3.9,2.5.1.94,4.1.2.42,4.2.1.33,2.3.1.277,2.4.2.54,4.4.1.24,4.2.1.55,2.7.7.68,2.4.2.57,True,1.0
4425,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47,True,0.918800,4.2.1.47,2.4.1.62,1.1.1.234,1.1.1.219,1.2.1.44,2.4.2.4,4.1.1.95,1.14.13.82,6.3.2.20,2.4.1.270,3.7.1.12,2.7.1.157,4.1.1.105,4.1.99.5,3.6.1.64,5.1.3.15,1.14.13.25,5.1.3.26,2.1.1.179,1.1.1.308,True,1.0


In [583]:
aa[aa.ec_number_blast.isnull()]

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,emzyme_pred_final,emzyme_pred_final_prob
421,P0DPM7,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.185797,2.5.1.18,5.2.1.8,6.1.1.6,1.3.7.6,1.8.4.11,3.6.1.55,2.3.1.248,3.4.21.32,1.2.1.13,4.2.3.98,1.14.11.33,3.6.1.25,1.13.11.16,1.3.3.3,5.1.99.8,1.11.1.11,4.1.3.40,2.7.8.37,1.15.1.1,3.4.17.6,False,0.167218
440,P0DSG0,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.037198,1.14.17.4,2.3.1.48,1.8.7.2,1.14.11.11,3.4.17.17,2.1.1.282,5.1.3.13,2.3.1.259,4.1.1.63,3.6.1.7,5.2.1.4,3.6.1.62,2.5.1.25,2.7.7.96,3.6.1.56,4.3.2.6,1.3.99.17,2.3.2.23,1.14.20.4,5.4.99.23,False,0.033478
559,P0CE63,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.171411,3.5.4.16,1.13.11.11,4.4.1.17,2.1.1.148,3.2.2.21,3.6.1.52,2.5.1.25,3.6.1.67,3.1.21.4,3.4.22.39,2.7.3.3,3.5.5.8,3.1.13.5,1.8.3.2,3.1.26.5,4.2.1.94,2.1.1.318,5.4.99.5,1.14.11.45,1.14.15.9,False,0.154270
577,P0CE58,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.171411,3.5.4.16,1.13.11.11,4.4.1.17,2.1.1.148,3.2.2.21,3.6.1.52,2.5.1.25,3.6.1.67,3.1.21.4,3.4.22.39,2.7.3.3,3.5.5.8,3.1.13.5,1.8.3.2,3.1.26.5,4.2.1.94,2.1.1.318,5.4.99.5,1.14.11.45,1.14.15.9,False,0.154270
770,P0DPN3,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.014556,7.1.1.2,4.1.3.40,4.4.1.20,3.1.13.5,1.17.4.4,3.1.26.5,2.5.1.18,5.4.99.5,1.14.11.45,3.6.1.64,7.2.4.2,4.4.1.26,3.6.1.55,1.14.99.48,3.3.2.11,2.1.1.179,2.4.1.257,2.4.1.132,3.8.1.9,2.7.8.37,False,0.013100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4358,P0DSF4,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.014341,3.1.26.5,6.1.1.5,5.6.2.1,2.3.1.197,2.7.7.6,1.11.1.1,1.13.11.6,3.2.2.23,1.8.4.12,4.7.1.1,4.1.99.11,2.3.2.31,4.2.99.18,1.14.13.149,1.8.7.2,6.4.1.8,3.1.12.1,2.1.1.355,4.1.99.23,1.8.4.14,False,0.012907
4372,P0DSE3,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.151827,1.14.20.4,1.10.3.15,1.8.4.11,3.5.5.8,2.7.1.68,2.3.2.24,2.3.2.23,2.3.1.48,4.2.3.98,2.3.1.259,3.6.1.62,3.5.4.34,1.14.11.11,4.4.1.17,3.5.1.52,5.4.99.43,1.3.7.2,3.1.21.4,5.2.1.4,3.6.1.8,False,0.136644
4386,P0DPN1,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.026671,2.1.1.86,1.14.14.18,2.3.1.135,2.4.1.132,2.4.1.257,2.5.1.103,4.2.1.78,3.3.2.11,3.4.21.98,1.14.13.148,1.14.12.18,2.7.1.216,3.4.17.6,2.4.2.45,1.3.99.30,1.14.14.73,3.2.1.101,3.5.1.92,3.6.1.55,4.2.3.126,False,0.024004
4402,P0DSG3,NaN,NaN,NaN,nan,NaN,NaN,NaN,False,0.015993,7.1.1.2,4.1.3.40,1.14.13.105,2.5.1.103,3.6.1.67,5.3.1.32,1.14.99.48,3.4.22.39,2.3.2.6,2.3.1.135,1.14.11.33,5.4.99.20,5.1.99.7,1.3.7.6,4.4.1.17,3.1.12.1,3.1.21.4,3.6.1.55,3.5.1.135,3.3.2.11,False,0.014394


In [622]:
def get_ec_rep(balstpred, slicepred, isenzyme, isenzymeprob):
    
    #非酶
    if (isenzyme=='False') and (isenzymeprob ==1):
        return ['', '', '', '','' ,1.0]
    
    #非酶但可能是酶
    if (isenzyme=='False') and (isenzymeprob !=1):
        return [slicepred.values[0], slicepred.values[1], slicepred.values[2], slicepred.values[3], slicepred.values[4] ,0.6]
    
    # 全一致
    if balstpred in slicepred.values:
        return [balstpred, '', '', '','' ,1.0]
    
    # 有比对结果
    if str(balstpred)!='NaN':
        return [balstpred, slicepred.values[0], slicepred.values[1],'' ,'' , 0.95]
    
    # 无比对结果
    if str(balstpred)=='NaN':
        return [slicepred.values[0], slicepred.values[1], slicepred.values[2], slicepred.values[3], slicepred.values[4] , 0.7]
    

In [623]:
list_enzyme_pred = []
for index, row in aa.iterrows():
    list_enzyme_pred+=[get_ec_rep(row.ec_number_blast, row[10:30] , row.emzyme_pred_final, row.emzyme_pred_final_prob)]

In [626]:
apredslice = pd.DataFrame(list_enzyme_pred)
apredslice

,0,1,2,3,4,5
0,3.6.5.-,3.6.5.2,1.18.6.1,,,0.95
1,3.1.3.2,,,,,1.00
2,3.1.13.1,,,,,1.00
3,3.6.1.-,2.9.1.3,2.7.4.33,,,0.95
4,7.1.1.7,,,,,1.00
...,...,...,...,...,...,...
4422,,,,,,1.00
4423,,,,,,1.00
4424,2.1.2.10,,,,,1.00
4425,4.2.1.47,,,,,1.00


In [627]:
aa['ec_pred_final_top1'] = apredslice[0]
aa['ec_pred_final_top2'] = apredslice[1]
aa['ec_pred_final_top3'] = apredslice[2]
aa['ec_pred_final_top4'] = apredslice[3]
aa['ec_pred_final_top5'] = apredslice[4]
aa['ec_pred_final_top1_prob'] = apredslice[5]

In [641]:
aa[(aa.ec_number.isnull()) & (aa.emzyme_pred_final=='True')]

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,emzyme_pred_final,emzyme_pred_final_prob,ec_pred_final_top1,ec_pred_final_top2,ec_pred_final_top3,ec_pred_final_top4,ec_pred_final_top5,ec_pred_final_top1_prob
14,P37690,NaN,NaN,100.0,False,False,1.0,-,True,0.880858,3.4.24.75,3.4.14.13,1.5.99.13,4.3.1.27,2.3.1.157,2.3.1.203,3.4.21.22,2.7.7.23,3.4.21.7,3.2.1.81,3.4.21.68,2.3.1.210,3.4.21.2,3.4.21.4,3.4.21.109,3.1.1.65,3.4.17.13,1.14.15.15,3.4.22.32,2.7.1.45,True,0.440429,-,3.4.24.75,3.4.14.13,,,0.95
28,P77737,NaN,NaN,100.0,False,False,1.0,-,True,0.727023,7.4.2.9,7.4.2.10,2.7.1.21,7.2.2.11,7.6.2.14,7.3.2.2,1.2.7.8,1.12.98.4,4.2.99.18,3.6.1.72,3.6.1.71,1.3.1.2,7.2.2.2,3.1.12.1,4.1.99.23,1.8.98.6,1.12.98.1,1.2.7.7,1.3.5.1,1.4.99.5,True,0.363512,-,7.4.2.9,7.4.2.10,,,0.95
118,P52130,NaN,NaN,100.0,False,False,1.0,-,True,0.635598,3.5.4.16,2.7.7.6,4.1.1.50,2.7.4.6,2.1.1.148,5.5.1.6,3.1.21.4,2.3.1.204,1.17.4.1,2.7.7.7,1.16.3.1,2.3.1.133,1.12.1.2,3.1.4.37,2.3.1.33,5.4.99.43,3.6.1.58,2.7.7.96,5.4.99.29,3.6.1.64,True,0.317799,-,3.5.4.16,2.7.7.6,,,0.95
129,P10346,NaN,NaN,100.0,False,False,1.0,-,True,0.661911,7.3.2.1,7.6.2.15,7.4.2.1,7.4.2.12,7.4.2.7,7.3.2.2,7.6.2.14,7.6.2.12,7.6.2.3,7.6.2.2,7.2.2.2,7.2.2.11,7.6.2.7,7.2.2.16,7.5.2.8,7.5.2.11,7.5.2.9,7.6.2.8,7.6.2.13,2.7.8.12,True,0.330956,-,7.3.2.1,7.6.2.15,,,0.95
162,P0AG76,NaN,NaN,100.0,False,False,1.0,-,True,0.662492,2.1.2.5,4.3.1.4,3.1.4.12,3.1.4.11,3.1.1.3,3.1.4.39,3.4.13.19,2.7.1.164,2.1.1.228,5.5.1.2,4.1.1.19,3.4.22.34,2.7.7.59,4.1.1.61,2.7.7.78,1.3.8.11,1.21.4.3,2.7.4.8,3.4.17.21,1.21.4.4,True,0.331246,-,2.1.2.5,4.3.1.4,,,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4380,P36928,NaN,NaN,100.0,False,False,1.0,-,True,0.778177,2.7.1.33,2.7.2.7,1.3.7.8,2.3.1.234,2.7.1.30,2.7.1.14,3.1.22.4,2.7.1.58,2.5.1.131,2.7.1.1,2.7.1.2,2.7.1.188,2.7.1.17,2.7.1.63,2.7.1.53,3.2.1.183,2.7.2.1,2.7.1.51,3.5.1.124,2.7.1.215,True,0.389088,-,2.7.1.33,2.7.2.7,,,0.95
4381,P52123,NaN,NaN,100.0,False,False,1.0,-,True,0.621218,4.3.2.1,2.5.1.1,3.1.26.3,1.3.2.3,2.7.7.19,2.5.1.10,2.1.1.218,3.4.22.45,2.3.2.23,2.4.1.198,3.1.7.10,2.7.7.42,2.7.1.158,4.2.3.54,4.1.99.13,3.4.22.68,4.2.3.75,2.7.7.89,6.4.1.2,4.2.3.76,True,0.310609,-,4.3.2.1,2.5.1.1,,,0.95
4388,Q2EER5,NaN,NaN,100.0,False,False,1.0,-,True,0.533342,1.1.1.37,2.1.1.113,2.7.1.197,2.7.1.191,1.8.99.2,6.5.1.2,1.16.1.7,2.7.7.2,2.1.1.67,4.3.1.17,2.4.2.9,5.4.99.18,1.18.1.2,2.7.4.27,2.7.11.32,1.14.19.9,2.7.1.205,2.1.1.211,5.3.1.34,1.20.4.1,True,0.266671,-,1.1.1.37,2.1.1.113,,,0.95
4397,P64612,NaN,NaN,100.0,False,False,1.0,-,True,0.500412,7.1.2.2,3.6.4.12,3.6.4.6,2.7.1.21,6.6.1.1,7.2.2.1,2.3.1.193,3.1.3.32,3.1.21.5,2.4.1.141,6.6.1.2,7.4.2.4,2.7.4.13,2.7.1.163,2.1.1.160,3.1.21.3,1.4.1.17,1.14.14.8,3.4.22.44,2.3.1.38,True,0.250206,-,7.1.2.2,3.6.4.12,,,0.95


In [642]:
aa[(~aa.ec_number.isnull()) & (aa.emzyme_pred_final=='False')]

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,emzyme_pred_final,emzyme_pred_final_prob,ec_pred_final_top1,ec_pred_final_top2,ec_pred_final_top3,ec_pred_final_top4,ec_pred_final_top5,ec_pred_final_top1_prob
261,P10408,7.4.2.8,NaN,100.0,False,False,1.0,-,False,0.002500,2.1.1.354,2.1.1.355,3.2.2.23,4.2.99.18,2.1.1.368,3.1.12.1,1.12.98.4,2.1.1.366,4.1.99.23,3.8.1.2,3.6.4.13,3.1.3.48,4.2.1.136,1.2.7.8,3.6.1.72,3.6.1.71,1.5.99.14,4.99.1.4,1.2.7.7,2.7.1.83,False,1.0,,,,,,1.0
1975,P0CF87,1.20.4.1,NaN,100.0,False,False,1.0,-,False,0.200937,1.20.4.1,3.5.4.38,1.11.1.17,3.5.4.36,3.5.4.5,1.20.4.3,1.8.1.8,1.21.98.1,2.3.2.29,1.3.4.1,5.4.3.2,1.8.7.3,3.1.3.64,1.13.11.15,1.17.99.6,1.1.1.363,1.8.2.3,1.11.1.25,3.5.4.12,1.14.13.237,False,1.0,,,,,,1.0
2304,P76556,2.7.2.1,NaN,100.0,False,False,1.0,-,False,0.331959,2.7.1.48,2.7.1.22,2.7.1.21,2.7.4.33,2.7.1.173,7.3.2.7,3.6.5.2,2.7.4.13,6.3.3.3,1.18.6.1,3.1.3.32,2.7.4.14,2.8.2.2,2.7.1.19,2.7.4.9,2.7.1.76,2.7.1.186,2.7.2.16,2.8.2.3,6.2.1.46,False,1.0,,,,,,1.0
3391,P45759,7.4.2.8,NaN,100.0,False,False,1.0,-,False,0.313598,3.6.4.12,5.6.1.1,3.6.4.13,2.7.1.21,6.3.2.13,7.2.2.1,2.3.1.193,2.3.2.27,3.6.4.6,2.4.1.227,6.3.2.7,6.6.1.1,3.1.21.5,1.17.99.6,6.3.2.9,2.7.8.12,2.7.7.62,1.8.98.5,2.7.1.156,1.3.7.11,False,1.0,,,,,,1.0


In [1]:
! python ../../baselines/deepec/deepec.py -i ./data/ecoli1515/1515.fasta -o ./data/ecoli1515/deepec

/home/shizhenkun/anaconda3/envs/deepec/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>
Licensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>
Check http://github.com/bbuchfink/diamond for updates.

#CPU threads: 1
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: ./data/ecoli1515/deepec
Opening the database...  [0.070168s]
#Target sequences to report alignments for: 25
Opening the input file...  [0.000316s]
Opening the output file...  [0.002264s]
Loading query sequences...  [0.004562s]
Masking queries...  [0.129646s]
Building query seed set...  [0.00518s]
Algorithm: Double-indexed
Building query histog

In [10]:
# 读取预测结果
deepec_results = pd.read_csv('./data/ecoli1515/deepec/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number_deepec'], header=0) #读入文件
deepec_results.ec_number_deepec=deepec_results.apply(lambda x: x['ec_number_deepec'].replace('EC:',''), axis=1)

In [11]:
deepec_results

,id,ec_number_deepec
0,P0AE22,3.1.3.2
1,P21499,3.1.13.1
2,P56100,1.10.3.10
3,P23874,2.7.11.1
4,P23837,2.7.13.3
...,...,...
1277,P39337,2.3.1.17
1278,P77806,2.6.1.88
1279,P0ADR8,3.2.2.10
1280,P27832,2.3.1.210


In [4]:
pred = pd.read_excel('../../results/1515/final_pred.xlsx')

In [13]:
aa=pred.merge(deepec_results, on='id', how='left')

In [15]:
aa[aa.ec_number == aa.ec_number_deepec]

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,...,13,14,15,16,17,18,19,emzyme_pred_final,emzyme_pred_final_prob,ec_number_deepec
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,0.0,1.0,3.1.3.2,True,0.586609,...,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,True,1.0,3.1.3.2
2,P21499,3.1.13.1,3.1.-.-,100.0,True,0.0,1.0,3.1.13.1,True,0.659836,...,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136,True,1.0,3.1.13.1
5,P23874,2.7.11.1,2.7.11.1,100.0,True,0.0,1.0,2.7.11.1,True,0.980370,...,5.3.1.9,5.4.99.43,3.8.1.10,2.1.1.200,1.14.13.69,3.5.99.2,1.14.13.172,True,1.0,2.7.11.1
12,P39304,4.1.1.85,4.1.2.-,100.0,True,0.0,1.0,4.1.1.85,True,0.998098,...,1.13.12.4,2.5.1.41,4.3.2.10,1.1.99.31,4.1.2.56,4.1.2.14,5.3.99.10,True,1.0,4.1.1.85
15,P0AD14,2.7.13.3,2.7.13.3,100.0,True,0.0,1.0,2.7.13.3,True,0.913177,...,1.14.13.154,1.14.14.76,1.14.19.41,1.14.14.16,1.14.14.124,1.14.14.125,1.14.19.50,True,1.0,2.7.13.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4483,P25718,3.2.1.1,3.2.1.1,100.0,True,0.0,1.0,3.2.1.1,True,0.986182,...,3.2.1.81,3.2.1.68,3.2.1.47,3.2.1.123,3.2.1.199,3.2.1.135,2.4.1.352,True,1.0,3.2.1.1
4491,P76330,2.7.7.65,2.7.7.65,100.0,True,0.0,1.0,2.7.7.65,True,0.881468,...,2.7.8.44,7.6.2.8,3.1.4.59,2.1.1.99,2.3.1.178,3.4.23.2,2.1.1.156,True,1.0,2.7.7.65
4492,P36655,1.8.1.8,1.8.1.8,100.0,True,0.0,1.0,1.8.1.8,True,0.788530,...,3.1.1.76,3.4.11.5,3.5.4.38,3.1.1.45,3.4.21.103,3.8.1.3,5.1.3.2,True,1.0,1.8.1.8
4495,P27248,2.1.2.10,2.1.2.10,100.0,True,0.0,1.0,2.1.2.10,True,0.983754,...,4.2.1.33,2.3.1.277,2.4.2.54,4.4.1.24,4.2.1.55,2.7.7.68,2.4.2.57,True,1.0,2.1.2.10


In [14]:
aa.head(3)

,id,ec_number,blattner_ec_number,pident,isemzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,...,13,14,15,16,17,18,19,emzyme_pred_final,emzyme_pred_final_prob,ec_number_deepec
0,P42641,3.6.5.-,NaN,100.0,True,0.0,1.0,3.6.5.-,True,0.988359,...,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27,True,1.0,NaN
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,0.0,1.0,3.1.3.2,True,0.586609,...,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,True,1.0,3.1.3.2
2,P21499,3.1.13.1,3.1.-.-,100.0,True,0.0,1.0,3.1.13.1,True,0.659836,...,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136,True,1.0,3.1.13.1


In [16]:
aa.to_excel('../../results/1515/final_pred_with_deepec.xlsx', index=None)